<a href="https://colab.research.google.com/github/sandeeps02/Zomato-Restaurant-Clustering-And-Sentiment-Analysis/blob/main/Zomato_Restaurant_Clustering_And_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Zomato Restaurant Clustering And Sentiment Analysis



##### **Project Type**    - EDA/Unsupervised
##### **Contribution**    - Individual


# **Project Summary -**

Zomato is an Indian restaurant aggregator and food delivery start-up founded by Deepinder Goyal and Pankaj Chaddah in 2008 Zomato provides information, menus and user-reviews of restaurants, and also has food delivery options from partner restaurants in select cities, India is quite famous for its diverse multi cuisine available in a large number of restaurants and hotel resorts, which is reminiscent of unity in diversity, Restaurant business in India is always evolving More Indians are warming up to the idea of eating restaurant food whether by dining outside or getting food delivered. The growing number of restaurants in very state of India has been a motivation to inspect the data to get some insights, interesting facts and figures about the Indian food industry in each city. So, this project focuses on analysing the Zomato restaurant data for each city in India. The Project focuses on Customers and Company, you have to analyze the sentiments of the reviews given by the customer in the data and make some useful conclusions in the form of Visualizations. Also, cluster the zomato restaurants into different segments The data is vizualized as it becomes easy to analyse data at instant. The Analysis also solves sorte of the business cases that can directly help the customers finding the Best restaurant in their locality and for the company to grow up and work on the fields they are currently logging in. This could help in clustering the restaurants into segments. Also the data has valuable information around cuisine and costing which can be used in cost vs. benefit analysis Data could be used for sentiment analysis. Also the metadata of reviewers can be u used for identifying the critics in the industry.

# **GitHub Link -**

https://github.com/sandeeps02/Zomato-Restaurant-Clustering-And-Sentiment-Analysis



# **Problem Statement**


**Write Problem Statement Here.**

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# Introduction to the Notebook

In this notebook we can explore the activities of the Zomato restaurants in Hyderabad, the capital and largest city of the Indian state of Telangana and the the jure capital of Andhra Pradesh.
We are going to work with two dataset:
* **Restaurant names and Metadata** - This could help in clustering the restaurants into segments. Also the data has valuable information around cuisine and costing which can be used in cost vs. benefit analysis
* **Restaurant reviews** - Data could be used for sentiment analysis. Also the metadata of reviewers can be used for identifying the critics in the industry.

# **Importing libraries and reading data files**

**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

**Reading datasets**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
name_filepath = '/content/drive/MyDrive/Colab Notebooks/Zomato dataset/Zomato Restaurant names and Metadata.csv'
reviews_filepath = '/content/drive/MyDrive/Colab Notebooks/Zomato dataset/Zomato Restaurant reviews.csv'

name_metadata_df = pd.read_csv(name_filepath)
reviews_df = pd.read_csv(reviews_filepath)

In [ ]:
restaurants = name_metadata_df.copy()
reviews = reviews_df.copy()

------------------------------------------------------------------------------------------------------

# **Restaurants dataset preprocessing**

**How big is the restaurants dataset?**

In [ ]:
restaurants.shape

**Let's have a look on the fisrt 5 rows of the dataset**

In [ ]:
restaurants.head()

**Convert the 'Cost' column, deleting the comma and changing the data type into 'int64'**

In [ ]:
restaurants['Cost'] = restaurants['Cost'].str.replace(",","").astype('int64')

**We convert the 'Cost' column data type but, what are the other features data type?**

In [ ]:
restaurants.info()

**Finally, how much missing values there are in these dataset?**

In [ ]:
restaurants.isnull().sum()

--------------------------------------------------------------------------------------------------

# **Reviews dataset preprocessing**

**How big is the reviews dataset?**

In [ ]:
reviews.shape

**Let's have a look on the fisrt 5 rows of the dataset**

In [ ]:
reviews.head()

In [ ]:
reviews.isnull().sum()

**As we can see, there are few missing values compared to the shape of the dataset so I decide to drop them all because there isn't a big loss**

In [ ]:
reviews.dropna(inplace = True)

In [ ]:
reviews.info()

**After have had a look on the type of the features, I want to change a few of them. First of all i want to change the 'Rating' column dtype.**

In [ ]:
reviews['Rating'].value_counts()

**As we can see there is that 'like' which is inconvertible to a number so the only way I can convert this functionality to type 'float64' is to omit that data**

In [ ]:
reviews=reviews[reviews['Rating']!='Like']

**Now, as said before i change a lot of dtype of the other features. I also add a new feature called 'Year' extrapolating the data from the 'Time' column**

In [ ]:
reviews['Rating'] = reviews['Rating'].astype('float64')
reviews['Reviews'],reviews['Followers']=reviews['Metadata'].str.split(',').str
reviews['Reviews'] = pd.to_numeric(reviews['Reviews'].str.split(' ').str[0])
reviews['Followers'] = pd.to_numeric(reviews['Followers'].str.split(' ').str[1])
reviews['Time']=pd.to_datetime(reviews['Time'])
reviews['Year'] = pd.DatetimeIndex(reviews['Time']).year
reviews['Hour'] = pd.DatetimeIndex(reviews['Time']).hour
reviews = reviews.drop(['Metadata'], axis = 1)
reviews.dtypes

**And finally we can have another look of the dataset with our changes**

In [ ]:
reviews.head()

---------------------------------------------------------------------------------------------------------

# Analysis of various themes of the datasets

# Cuisines variety analisys

**First of all we can see the 10 cuisines most presents in our dataset**

In [ ]:
# Split cuisines and count occurrences
cuis_list = {}
for cuisines in restaurants['Cuisines']:
    for cuisine in cuisines.split(', '):
        if cuisine in cuis_list:
            cuis_list[cuisine] += 1
        else:
            cuis_list[cuisine] = 1

# Create DataFrame and sort
cuis_df = pd.DataFrame({'Cuisine': list(cuis_list.keys()), 'Counts of Restaurants': list(cuis_list.values())})
cuis_df.sort_values(by='Counts of Restaurants', ascending=False, inplace=True)

# Select and print top 10 cuisines
top_10_cuis = cuis_df.head(10)
print('The Top 10 Cuisines are:\n', top_10_cuis)

**A list is a good summertime but a bar chart is perhaps a better way to represent a lot of data in a more intuitive way**

In [ ]:
plt.figure(figsize=(15, 10))

# Using Seaborn's bar plot for consistent styling
sns.barplot(data=cuis_df, x='Cuisine', y='Counts of Restaurants', palette="crest")

plt.xlabel('Cuisines', size=15)
plt.xticks(rotation=45, ha="right")  # Adjust rotation and alignment for readability
plt.ylabel('Cuisine available at Number of Restaurants', size=15)
plt.title('Most popular cuisines at Restaurants', size=30, color='green')

plt.tight_layout()
plt.show()

**Secondly we can draw a WordCloud of the most nominated words in the cuisine feature**

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Assuming you have 'cuis_list' dictionary with cuisine counts

words_list = cuis_list.keys()
strr = ' '.join(words_list)  # Join the cuisine names using a space

wordcloud = WordCloud(width=1400, height=1400,
                      background_color='ivory',
                      min_font_size=12).generate(strr)

# plot the WordCloud image
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud, interpolation='bilinear')  # Use bilinear interpolation for better quality
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()


# Costs of the restaurants

**Now we can plot a graph representing the restaurants based on the costs of them**

In [ ]:
restaurants_cost=restaurants.groupby('Name').apply(lambda x:np.average(x['Cost'])).reset_index(name='Cost')
restaurants_cost.sort_values(by='Cost',ascending=False,inplace=True)
avg=np.average(restaurants_cost['Cost'])
plt.figure(figsize=(25, 10))

# Using Seaborn's bar plot for consistent styling
sns.barplot(data=restaurants_cost, x='Name', y='Cost', palette="viridis")

plt.scatter(restaurants_cost['Name'], np.full_like(restaurants_cost['Name'], avg), color='red', label='Average Cost at Restaurant')

plt.xlabel('Restaurants', size=15)
plt.xticks(rotation=90)
plt.ylabel('Average Cost', size=15)
plt.title('Overall Cost Summary of Restaurants', size=30)
plt.legend()

plt.tight_layout()
plt.show()

**But which are the most expensive restaurants?**

In [ ]:
best_5_rest = restaurants_cost[:5]
best_5_rest.sort_values(by='Cost',ascending=True,inplace=True)
plt.figure(figsize=(20,5))
plt.bar(best_5_rest['Name'],best_5_rest['Cost'], color = sns.color_palette("hls", 8))
plt.title('The 5 most expensive Restaurants',size=28)
plt.xlabel('Restaurants',size=15)
plt.ylabel('Average Cost',size=15)

**And which are the cheapest restaurants?**

In [ ]:
worst_5_rest = restaurants_cost[-5:]
plt.figure(figsize=(20,5))
plt.bar(worst_5_rest['Name'],worst_5_rest['Cost'], color = sns.color_palette("hls", 8))
plt.title('The 5 most cheap Restaurants',size=28)
plt.xlabel('Restaurants',size=15)
plt.ylabel('Average Cost',size=15)

**Finally, we can draw a WordCloud of the most nominated words in the 'Name' feature**

In [ ]:
Rests = restaurants.Name.unique()
rest_string = ' '.join(Rests)  # Join the restaurant names using a space

wordcloud = WordCloud(width=1400, height=1400,
                      background_color='lavenderblush',
                      min_font_size=12).generate(rest_string)

# Plot the WordCloud image
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

# Top 15 Reviewer

**First of all, we extrapolate the 15 profiles that have made more reviews**

In [ ]:
reviewer_list = reviews.groupby('Reviewer').apply(lambda x: x['Reviewer'].count()).reset_index(name='Review Count')
reviewer_list = reviewer_list.sort_values(by = 'Review Count',ascending=False)
top_reviewers = reviewer_list[:15]

**Secondly, we plot a graph to represent them**

In [ ]:
plt.figure(figsize=(13, 5))

# Using Seaborn's bar plot for consistent styling
sns.barplot(data=top_reviewers, x='Reviewer', y='Review Count', palette="hls")

plt.xticks(rotation=75, ha='right')  # Adjust rotation and alignment for readability
plt.title('Top 15 Reviewers', size=28)
plt.xlabel('Name (or Nickname)', size=15)
plt.ylabel('Number of Reviews', size=15)

plt.tight_layout()
plt.show()

**Then we can calculate the average of their ratings review**

In [ ]:
review_ratings=reviews.groupby('Reviewer').apply(lambda x:np.average(x['Rating'])).reset_index(name='Average Ratings')
review_ratings=pd.merge(top_reviewers,review_ratings,how='inner',left_on='Reviewer',right_on='Reviewer')
top_reviewers_ratings=review_ratings[:15]

In [ ]:
top_reviewers_ratings

**And also plot a line chart to view them in a simple way**

In [ ]:
review_ratings_plot = top_reviewers_ratings.groupby('Review Count').apply(lambda x:np.average(x['Average Ratings'])).reset_index(name='Average')
plt.figure(figsize=(8, 8))

# Plotting the line plot and scatter plot
plt.plot(review_ratings_plot['Review Count'], review_ratings_plot['Average'], label='Average Ratings')
plt.scatter(review_ratings_plot['Review Count'], review_ratings_plot['Average'], color='red', label='Data Points')

plt.xlabel('Number of Reviews by an User', size=15)
plt.ylabel('Average Ratings per Review Submitted', size=15)
plt.title('Distribution of Average Ratings per Review Submitted', size=20)
plt.legend()  # Adding a legend to differentiate line plot and scatter points

plt.tight_layout()
plt.show()

# Big comparison: Followers vs Reviews vs Pictures

**How it has been the growth of the followers and the reviews in this 3 years of Zomato?**

In [ ]:
review_follow_1=reviews.groupby('Year').apply(lambda x:np.sum(x['Reviews'])).reset_index(name='Total Reviews')
review_follow_2=reviews.groupby('Year').apply(lambda x:np.max(x['Followers'])).reset_index(name='Total Followers')
pictures =reviews.groupby('Year').apply(lambda x:np.sum(x['Pictures'])).reset_index(name='Total Pictures')
review_follow=pd.merge(review_follow_1,review_follow_2, how='inner',left_on='Year',right_on='Year')
review_follow=pd.merge(review_follow, pictures, how='inner',left_on='Year',right_on='Year')
review_follow

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(review_follow['Year'],review_follow['Total Followers'], color = 'blue')
plt.plot(review_follow['Year'],review_follow['Total Reviews'], color = "green")
plt.plot(review_follow['Year'],review_follow['Total Pictures'], color = "orange")
plt.xlabel('Year',size=15)
plt.xticks(rotation=75)
plt.grid()
plt.ylabel('Reviews/Followers/Pictures',size=15)
plt.title('Reviews,Followers and Pictures count with Time',size=20)
plt.legend(['Total Followers','Total Review', 'Total Pictures'])

**As we can see there is a big difference between them, there are many more reviews than the total followers. Not to mention the images that have been considered by even fewer people, too bad because they could be an added value as they can entice customers more immediately than just written reviews.**

# Reviews per hour

**What is the period of time in a day what the people make more reviews?**

In [ ]:
reviews_for_hour = reviews.groupby('Hour').apply(lambda x: x['Hour'].count()).reset_index(name='Reviews per hour')
reviews_for_hour

In [ ]:
plt.figure(figsize=(15, 5))

# Using Seaborn's bar plot for consistent styling
sns.barplot(data=reviews_for_hour, x='Hour', y='Reviews per hour', palette="hls")

plt.title('Reviews per Hour', size=28)
plt.grid(True)  # Adding grid lines
plt.xlabel('Hours', size=15)
plt.ylabel('Number of Reviews', size=15)

plt.tight_layout()
plt.show()

**As we can se there is an increase of reviews made from the afternoon to the midnight and then in the morning there is a decrease, but I think this is normal because in during the morning the most of the people is working or is at school**

# Most common words in reviews

**What are the most common words in the reviews?**

**First of all we import the 'spacy' library that is one of the most popular library for NLP (Natural Language Processing)**

In [ ]:
import spacy

**And then we draw a WordCloud of the most common words after have found them with the tokenization method**

In [ ]:
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Load the English NLP model from spaCy
nlp = spacy.load('en_core_web_sm')

reviews_feature = reviews['Review']
all_words = []

# Process each review
for review in reviews_feature:
    doc = nlp(review)
    words = [token.text
             for token in doc
             if not token.is_stop and not token.is_punct]
    all_words.extend(words)

rest_string = ' '.join(all_words)  # Join processed words using a space

wordcloud = WordCloud(width=1400, height=1400,
                      background_color='lavenderblush',
                      min_font_size=12).generate(rest_string)

# Plot the WordCloud image
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()


# Costs vs Ratings

**Will the most expensive restaurants be the highest rated? Will high prices please customers with high quality food?**

In [ ]:
df_merged= reviews.merge(restaurants, how='inner', left_on='Restaurant', right_on='Name')
df_merged.head(1)

In [ ]:
plt.figure(figsize=(10, 6))

# Create a box plot using Seaborn's sns.boxplot
sns.boxplot(x=df_merged['Rating'], y=df_merged['Cost'])

plt.title('Box Plot of Ratings vs. Costs', size=18)
plt.xlabel('Rating', size=15)
plt.ylabel('Cost', size=15)

plt.tight_layout()
plt.show()

**As we can see there is not much data on expensive restaurants probably because many people prefer to spend less, but we can also see that the Rating 5.0 box is the most elongated vertically, so we can say that people do not go to eat in expensive restaurants. often but the few times they go there they can feel satisfied**

----------------------------------------------------

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***